In [2]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,count
from pyspark.sql.window import Window


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

24/06/16 20:35:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [9]:
students = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "students") \
    .option("user", "root") \
    .option("password", "password") \
    .load()


courses = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "courses") \
    .option("user", "root") \
    .option("password", "password") \
    .load()



enrollments = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "enrollments") \
    .option("user", "root") \
    .option("password", "password") \
    .load()



In [10]:
enrollments = enrollments\
        .withColumn("enr_cnt",count("student_id").over(Window.partitionBy(col("student_id"))))

courses = courses\
        .withColumn("major_cnt",count("major").over(Window.partitionBy(col("major"))))

In [14]:
enrollments\
        .join(courses,on=enrollments.course_id == courses.course_id)\
        .filter((col("grade") == 'A') & (col("enr_cnt") == col("major_cnt")))\
        .withColumn("final_std_cnt",count("student_id").over(Window.partitionBy(col("student_id"))))\
        .filter(col("major_cnt")==col("final_std_cnt"))\
        .select("student_id").distinct()\
        .show()

+----------+
|student_id|
+----------+
|         1|
|         3|
+----------+

